In [55]:
import pandas as pd
from datetime import datetime, timedelta
import json

In [53]:
with open("updated_grants_reviewed.json", "r") as f:
    grants = json.load(f)

In [ ]:
delegator_df = pd.read_csv("../data/delegator_df.csv")
voter_df = pd.read_csv("../data/voter_df.csv")

In [35]:
voter_df.columns

Index(['dt', 'block_timestamp', 'block_number', 'transaction_hash',
       'voter_address', 'proposal_id', 'decision', 'reason', 'voting_power'],
      dtype='object')

In [32]:
def query_delegates(df, start_date_str, addresses):

    filtered_df = df[df['delegated_to'].isin(addresses)]
    project_users = filtered_df['delegator'].dropna().unique()
    start_date = pd.to_datetime(start_date_str) 

    df['dt'] = pd.to_datetime(df['dt'])

    # only keep rows where the delegator is a project user and dt > start_date
    tx_filtered = df[
        (df['delegator'].isin(project_users)) & 
        (df['dt'] > start_date)
    ].copy()

    tx_filtered.sort_values(by=['delegator', 'dt'], inplace=True)

    # calculate lag (previous_tx_date)
    tx_filtered['previous_tx_date'] = tx_filtered.groupby('delegator')['dt'].shift(1)

    tx_filtered['date_diff'] = (tx_filtered['dt'] - tx_filtered['previous_tx_date']).dt.days

    # first seen if no prior tx or gap > 90 days
    rolling_first_seen = tx_filtered[
        tx_filtered['previous_tx_date'].isnull() | (tx_filtered['date_diff'] > 90)
    ][['delegator', 'dt']].rename(columns={'dt': 'first_seen_date'})

    date_series = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=pd.Timestamp.today())
    })

    daily_counts = (
        date_series
        .merge(rolling_first_seen, left_on='date', right_on='first_seen_date', how='left')
        .groupby('date')
        .delegator.nunique()
        .reset_index(name='unique_delegates')
    )

    daily_counts.sort_values('date', inplace=True)

    return daily_counts

In [75]:
for grant in grants:
    if "north_star" in grant.keys() and grant["north_star"] == "new_delegators":
        project_name = grant["project_name"]
        clean_name = project_name.lower().replace(" ", "_").replace(".", "-").replace("/","-")
        print(f"Working on {project_name}")

        initial_delivery = datetime.strptime(grant["meta"]["Initial Delivery Date"], "%Y-%m-%dT%H:%M:%S")

        # Calculate time passed since initial delivery date
        time_since = datetime.now() - initial_delivery

        # Go backwards that same amount of time
        start_date = initial_delivery - time_since
        start_date_str = start_date.strftime("%m-%d-%Y")

        curr_addresses = list(set(list(address.keys())[0] for address in grant['addresses']))

        delegator_result = query_delegates(delegator_df, start_date_str, curr_addresses)
        delegator_result["unique_voters"] = 0
        delegator_result.to_csv(f"data/{clean_name}/{clean_name}_delegators_and_voters.csv", index=False)

Working on Optimism GovQuests
Working on Event Horizon Public Access Voter Pool


In [38]:
def query_voters(df, addresses, start_date):

    start_date = pd.to_datetime(start_date)
    df['dt'] = pd.to_datetime(df['dt'])
    filtered_df = df[df['voter_address'].isin(addresses)]

    # filter for dt > start_date
    tx_filtered = filtered_df[filtered_df['dt'] > start_date].copy()

    # sort and compute lag by proposal_id
    tx_filtered.sort_values(by=['proposal_id', 'dt'], inplace=True)
    tx_filtered['previous_tx_date'] = tx_filtered.groupby('proposal_id')['dt'].shift(1)

    # first seen proposal_id if no prior tx or gap > 90 days
    tx_filtered['date_diff'] = (tx_filtered['dt'] - tx_filtered['previous_tx_date']).dt.days

    rolling_first_seen = tx_filtered[
        tx_filtered['previous_tx_date'].isnull() | (tx_filtered['date_diff'] > 90)
    ][['proposal_id', 'dt']].rename(columns={'dt': 'first_seen_date'})

    # create date series from start_date to today
    date_series = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=pd.Timestamp.today())
    })

    # count distinct proposal_id (voters) per day
    daily_counts = (
        date_series
        .merge(rolling_first_seen, left_on='date', right_on='first_seen_date', how='left')
        .groupby('date')
        .proposal_id.nunique()
        .reset_index(name='unique_voters')
    )

    return daily_counts